<a href="https://colab.research.google.com/github/RomanenkoAleksandr/demo/blob/main/spark_SQL_homework2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
pip install --quiet pyspark

In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Practice").getOrCreate()

In [6]:
from pyspark import SparkFiles


df = spark.read.option('inferSchema', 'true').option('header', 'true').csv('/content/drive/MyDrive/Colab Notebooks/covid-data.csv')

Задание 1

In [7]:
from pyspark.sql import functions as F

In [8]:
df.select('iso_code', 'location', (F.col('total_cases') / F.col('population') * 100 ).alias('percentage_of_cases')).where(F.col('date').endswith('03-31')).sort(F.col('percentage_of_cases').desc()).show(15)

+--------+-------------+-------------------+
|iso_code|     location|percentage_of_cases|
+--------+-------------+-------------------+
|     AND|      Andorra| 15.543907331909661|
|     MNE|   Montenegro| 14.523725364693293|
|     CZE|      Czechia| 14.308848404077997|
|     SMR|   San Marino| 13.937179562732041|
|     SVN|     Slovenia| 10.370805779121204|
|     LUX|   Luxembourg|  9.847342390123583|
|     ISR|       Israel|  9.625106044786802|
|     USA|United States|  9.203010995860707|
|     SRB|       Serbia|  8.826328557933492|
|     BHR|      Bahrain|  8.488860079114566|
|     PAN|       Panama|  8.228739065460761|
|     PRT|     Portugal|  8.058699735120369|
|     EST|      Estonia|  8.022681579659551|
|     SWE|       Sweden|  7.969744347858805|
|     LTU|    Lithuania|  7.938864728274825|
+--------+-------------+-------------------+
only showing top 15 rows



Задание 2

In [9]:
from pyspark.sql import Window

In [12]:
window = Window.partitionBy('location')

df_filtered = df.select('location', 'date', 'new_cases').where((F.col('date') >= '2021-03-25') & (F.col('date') <= '2021-03-31') & (~F.col('iso_code').startswith('OWID')))

df_filtered.select('location', 'date', F.max('new_cases').over(window).alias('max_new_cases')).where(F.col('new_cases') == F.col('max_new_cases')).sort(F.col('max_new_cases').desc()).show(10)

+-------------+----------+-------------+
|     location|      date|max_new_cases|
+-------------+----------+-------------+
|       Brazil|2021-03-25|     100158.0|
|United States|2021-03-26|      77321.0|
|        India|2021-03-31|      72330.0|
|       France|2021-03-31|      59054.0|
|       Turkey|2021-03-31|      39302.0|
|       Poland|2021-03-26|      35145.0|
|      Germany|2021-03-31|      25014.0|
|        Italy|2021-03-26|      24076.0|
|         Peru|2021-03-25|      19206.0|
|      Ukraine|2021-03-26|      18226.0|
+-------------+----------+-------------+
only showing top 10 rows



Задание 3

In [25]:
window = Window.partitionBy('location').orderBy('date')

df_filtered = df.select('*').where((F.col('iso_code') == 'RUS') & (F.col('date') >= '2021-03-25') & (F.col('date') <= '2021-03-31'))

df_filtered.select('date', F.lag('new_cases').over(window).alias('previous_day_new_cases'), 'new_cases', (F.col('new_cases') - F.col('previous_day_new_cases')).alias('delta')).show()

+----------+----------------------+---------+------+
|      date|previous_day_new_cases|new_cases| delta|
+----------+----------------------+---------+------+
|2021-03-25|                  NULL|   9128.0|  NULL|
|2021-03-26|                9128.0|   9073.0| -55.0|
|2021-03-27|                9073.0|   8783.0|-290.0|
|2021-03-28|                8783.0|   8979.0| 196.0|
|2021-03-29|                8979.0|   8589.0|-390.0|
|2021-03-30|                8589.0|   8162.0|-427.0|
|2021-03-31|                8162.0|   8156.0|  -6.0|
+----------+----------------------+---------+------+

